In [1]:
#Write a python program which searches all the product under a particular product from www.amazon.in. The
#product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for
#guitars.
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

def search_amazon(product_name):
    driver = webdriver.Chrome()  # Make sure you have chromedriver installed and in PATH
    driver.get("https://www.amazon.in")
    search_box = driver.find_element(By.ID, "twotabsearchtextbox")
    search_box.send_keys(product_name)
    search_box.send_keys(Keys.RETURN)
    time.sleep(3)  # Allow time for the page to load
    
    products = driver.find_elements(By.CSS_SELECTOR, "div[data-component-type='s-search-result']")
    
    if products:
        print("Products found for '{}' on Amazon India:".format(product_name))
        for product in products:
            product_title = product.find_element(By.CSS_SELECTOR, "span.a-text-normal").text
            product_price = product.find_element(By.CSS_SELECTOR, "span.a-price-whole").text
            print("-" * 50)
            print("Product: ", product_title)
            print("Price: ₹", product_price)
    else:
        print("No products found for '{}' on Amazon India".format(product_name))
    
    driver.quit()

if __name__ == "__main__":
    product_name = input("Enter the product name you want to search on Amazon India: ")
    search_amazon(product_name)

Enter the product name you want to search on Amazon India: guitar
Products found for 'guitar' on Amazon India:
--------------------------------------------------
Product:  Kadence rosewood Guitar Frontier Series, Electric Acoustic Black Guitar With EQ, Die Cast Keys, Set Of Strings, Strap, Picks And Bag (Black EQ, Electric Acoustic)
Price: ₹ 4,899
--------------------------------------------------
Product:  Kadence Frontier guitar with Online Guitar learning course , Wine Red Acoustic Guitar with Die Cast Keys, Set of Strings, Strap, Picks and Bag (Wine Red, Acoustic)
Price: ₹ 4,799
--------------------------------------------------
Product:  Kadence Slowhand Premium Jumbo Semi Acoustic Guitar with Heavy Padded Bag, guitar cable, Pro Capo (Black Spruce Wood)
Price: ₹ 9,999
--------------------------------------------------
Product:  MOCKING BIRD 40 inch Acoustic guitar with Bag, Belt, Strings set and 5 Picks guitar for beginners musical instrument Cutaway Design gitaar
Price: ₹ 5,489
-

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"span.a-price-whole"}
  (Session info: chrome=123.0.6312.123); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF66BBD7032+63090]
	(No symbol) [0x00007FF66BB42C82]
	(No symbol) [0x00007FF66B9DEC65]
	(No symbol) [0x00007FF66BA2499D]
	(No symbol) [0x00007FF66BA24ADC]
	(No symbol) [0x00007FF66BA1A0AC]
	(No symbol) [0x00007FF66BA4701F]
	(No symbol) [0x00007FF66BA1A00A]
	(No symbol) [0x00007FF66BA471F0]
	(No symbol) [0x00007FF66BA63412]
	(No symbol) [0x00007FF66BA46D83]
	(No symbol) [0x00007FF66BA183A8]
	(No symbol) [0x00007FF66BA19441]
	GetHandleVerifier [0x00007FF66BFD25AD+4238317]
	GetHandleVerifier [0x00007FF66C00F70D+4488525]
	GetHandleVerifier [0x00007FF66C0079EF+4456495]
	GetHandleVerifier [0x00007FF66BCB0576+953270]
	(No symbol) [0x00007FF

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

def scrape_product_details(driver, product):
    try:
        brand_name = product.find_element(By.CSS_SELECTOR, "span.a-size-base-plus.a-color-base").text
    except:
        brand_name = "-"
    
    try:
        product_name = product.find_element(By.CSS_SELECTOR, "span.a-size-base-plus.a-color-base.a-text-normal").text
    except:
        product_name = "-"
    
    try:
        price = product.find_element(By.CSS_SELECTOR, "span.a-price-whole").text
    except:
        price = "-"
    
    try:
        return_exchange = product.find_element(By.CSS_SELECTOR, "div.a-row.a-size-base.a-color-secondary").text
    except:
        return_exchange = "-"
    
    try:
        expected_delivery = product.find_element(By.CSS_SELECTOR, "span.a-text-bold").text
    except:
        expected_delivery = "-"
    
    try:
        availability = product.find_element(By.CSS_SELECTOR, "span.a-size-base.a-color-success").text
    except:
        availability = "-"
    
    try:
        product_url = product.find_element(By.CSS_SELECTOR, "a.a-link-normal.a-text-normal").get_attribute('href')
    except:
        product_url = "-"
    
    return {
        "Brand Name": brand_name,
        "Name of the Product": product_name,
        "Price": price,
        "Return/Exchange": return_exchange,
        "Expected Delivery": expected_delivery,
        "Availability": availability,
        "Product URL": product_url
    }

def search_and_scrape_amazon(product_name, pages):
    driver = webdriver.Chrome()  # Make sure you have chromedriver installed and in PATH
    driver.get("https://www.amazon.in")
    search_box = driver.find_element(By.ID, "twotabsearchtextbox")
    search_box.send_keys(product_name)
    search_box.send_keys(Keys.RETURN)
    time.sleep(3)  # Allow time for the page to load
    
    products_data = []
    
    for page in range(1, pages + 1):
        products = driver.find_elements(By.CSS_SELECTOR, "div[data-component-type='s-search-result']")
        
        for product in products:
            product_data = scrape_product_details(driver, product)
            products_data.append(product_data)
        
        try:
            next_page_button = driver.find_element(By.CSS_SELECTOR, ".a-last")
            next_page_button.click()
            time.sleep(2)  # Allow time for the page to load
        except:
            break
    
    df = pd.DataFrame(products_data)
    df.to_csv("amazon_products.csv", index=False)
    driver.quit()

if __name__ == "__main__":
    product_name = input("Enter the product name you want to search on Amazon India: ")
    search_and_scrape_amazon(product_name, pages=3)
def scrape_product_details(driver, product):
    try:
        brand_name = product.find_element(By.CSS_SELECTOR, "span.a-size-base-plus.a-color-base").text
    except:
        brand_name = "-"
    
    try:
        product_name = product.find_element(By.CSS_SELECTOR, "span.a-size-base-plus.a-color-base.a-text-normal").text
    except:
        product_name = "-"
    
    try:
        price = product.find_element(By.CSS_SELECTOR, "span.a-price-whole").text
    except:
        price = "-"
    
    try:
        return_exchange = product.find_element(By.CSS_SELECTOR, "div.a-row.a-size-base.a-color-secondary").text
    except:
        return_exchange = "-"
    
    try:
        expected_delivery = product.find_element(By.CSS_SELECTOR, "span.a-text-bold").text
    except:
        expected_delivery = "-"
    
    try:
        availability = product.find_element(By.CSS_SELECTOR, "span.a-size-base.a-color-success").text
    except:
        availability = "-"
    
    try:
        product_url = product.find_element(By.CSS_SELECTOR, "a.a-link-normal.a-text-normal").get_attribute('href')
    except:
        product_url = "-"
    
    return {
        "Brand Name": brand_name,
        "Name of the Product": product_name,
        "Price": price,
        "Return/Exchange": return_exchange,
        "Expected Delivery": expected_delivery,
        "Availability": availability,
        "Product URL": product_url
    }

def search_and_scrape_amazon(product_name, pages):
    driver = webdriver.Chrome()  # Make sure you have chromedriver installed and in PATH
    driver.get("https://www.amazon.in")
    search_box = driver.find_element(By.ID, "twotabsearchtextbox")
    search_box.send_keys(product_name)
    search_box.send_keys(Keys.RETURN)
    time.sleep(3)  # Allow time for the page to load
    
    products_data = []
    
    for page in range(1, pages + 1):
        products = driver.find_elements(By.CSS_SELECTOR, "div[data-component-type='s-search-result']")
        
        for product in products:
            product_data = scrape_product_details(driver, product)
            products_data.append(product_data)
        
        try:
            next_page_button = driver.find_element(By.CSS_SELECTOR, ".a-last")
            next_page_button.click()
            time.sleep(2)  # Allow time for the page to load
        except:
            break
    
    df = pd.DataFrame(products_data)
    df.to_csv("amazon_products.csv", index=False)
    driver.quit()

if __name__ == "__main__":
    product_name = input("Enter the product name you want to search on Amazon India: ")
    search_and_scrape_amazon(product_name, pages=3)


Enter the product name you want to search on Amazon India: guitar
Enter the product name you want to search on Amazon India: jeans


In [3]:
#Write a python program to access the search bar and search button on images.google.com and scrape 10
#images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’. 
import os
import requests
from bs4 import BeautifulSoup

def scrape_images(keyword, num_images):
    driver = webdriver.Chrome()  # Make sure you have chromedriver installed and in PATH
    driver.get("https://www.google.com/imghp")
    
    search_bar = driver.find_element(By.NAME, "q")
    search_bar.send_keys(keyword)
    search_bar.send_keys(Keys.RETURN)
    time.sleep(2)  # Allow time for the page to load
    
    images = []
    while len(images) < num_images:
        # Scroll to load more images
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for more images to load
        
        # Extract image URLs
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        img_tags = soup.find_all('img', class_='rg_i')
        for img_tag in img_tags:
            try:
                img_url = img_tag['data-src']
                if img_url.startswith("http"):
                    images.append(img_url)
                    if len(images) == num_images:
                        break
            except KeyError:
                pass

        # Clicking on the last thumbnail to load more images
        try:
            driver.find_element(By.CSS_SELECTOR, ".mye4qd").click()
        except:
            break

    # Create directory if it doesn't exist
    directory = 'images_' + keyword
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Download images
    for i, image_url in enumerate(images):
        response = requests.get(image_url)
        with open(os.path.join(directory, f"{keyword}_{i+1}.jpg"), 'wb') as file:
            file.write(response.content)
    
    print(f"Downloaded {len(images)} images for '{keyword}'")
    
    driver.quit()

if __name__ == "__main__":
    keywords = ['fruits', 'cars', 'Machine Learning', 'Guitar', 'Cakes']
    num_images_per_keyword = 10
    for keyword in keywords:
        scrape_images(keyword, num_images_per_keyword)


Downloaded 10 images for 'fruits'
Downloaded 0 images for 'cars'
Downloaded 0 images for 'Machine Learning'
Downloaded 0 images for 'Guitar'
Downloaded 0 images for 'Cakes'


In [6]:
#Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com
#and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand
#Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,
#“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the
#details is missing then replace it by “- “. Save your results in a dataframe and CSV. 


def scrape_smartphone_details(driver):
    smartphones = driver.find_elements(By.CSS_SELECTOR, "div._1AtVbE")
    smartphone_details = []
    for smartphone in smartphones:
        details = {}
        try:
            details["Brand Name"] = smartphone.find_element(By.CSS_SELECTOR, "div._4rR01T").text
        except:
            details["Brand Name"] = "-"
        
        try:
            details["Smartphone Name"] = smartphone.find_element(By.CSS_SELECTOR, "a._2rpwqI").text
        except:
            details["Smartphone Name"] = "-"
        
        try:
            details["Colour"] = smartphone.find_element(By.CSS_SELECTOR, "a._1Wb-Sx").text
        except:
            details["Colour"] = "-"
        
        features = smartphone.find_elements(By.CSS_SELECTOR, "li.tVe95H")
        for feature in features:
            text = feature.text
            if "RAM" in text:
                details["RAM"] = text.split("|")[0].strip()
            elif "ROM" in text:
                details["Storage(ROM)"] = text.split("|")[0].strip()
            elif "Primary Camera" in text:
                details["Primary Camera"] = text.split("|")[0].strip()
            elif "Secondary Camera" in text:
                details["Secondary Camera"] = text.split("|")[0].strip()
            elif "Display Size" in text:
                details["Display Size"] = text.split("|")[0].strip()
            elif "Battery Capacity" in text:
                details["Battery Capacity"] = text.split("|")[0].strip()
        
        try:
            details["Price"] = smartphone.find_element(By.CSS_SELECTOR, "div._30jeq3._1_WHN1").text
        except:
            details["Price"] = "-"
        
        try:
            details["Product URL"] = smartphone.find_element(By.CSS_SELECTOR, "a._1fQZEK").get_attribute('href')
        except:
            details["Product URL"] = "-"
        
        smartphone_details.append(details)
    
    return smartphone_details

def search_and_scrape_flipkart(product_name):
    driver = webdriver.Chrome()  # Make sure you have chromedriver installed and in PATH
    driver.get("https://www.flipkart.com")
    
    # Close the login popup if it appears
    try:
        driver.find_element(By.CSS_SELECTOR, "button._2KpZ6l._2doB4z").click()
    except:
        pass
    
    search_bar = driver.find_element(By.CSS_SELECTOR, "input[type='text']")
    search_bar.send_keys(product_name)
    search_bar.send_keys(Keys.RETURN)
    time.sleep(3)  # Allow time for the page to load
    
    smartphone_details = scrape_smartphone_details(driver)
    df = pd.DataFrame(smartphone_details)
    df.to_csv("flipkart_smartphones.csv", index=False)
    
    #driver.quit()

if __name__ == "__main__":
    product_name = input("Enter the smartphone you want to search on Flipkart: ")
    search_and_scrape_flipkart(product_name)


Enter the smartphone you want to search on Flipkart: oppo


In [7]:
def scrape_coordinates(city):
    driver = webdriver.Chrome()  # Make sure you have chromedriver installed and in PATH
    driver.get("https://www.google.com/maps")
    
    search_box = driver.find_element(By.ID, "searchboxinput")
    search_box.send_keys(city)
    search_box.send_keys(Keys.RETURN)
    time.sleep(3)  # Allow time for the page to load
    
    try:
        url = driver.current_url
        if "@," in url:
            coordinates = url.split("@")[1].split(",")[0:2]
            latitude = coordinates[0]
            longitude = coordinates[1]
            print(f"Latitude: {latitude}, Longitude: {longitude}")
        else:
            print("Coordinates not found for the specified city.")
    except:
        print("Error occurred while scraping coordinates.")
    
    driver.quit()

if __name__ == "__main__":
    city = input("Enter the city name to search for its coordinates on Google Maps: ")
    scrape_coordinates(city)


Enter the city name to search for its coordinates on Google Maps: Pune
Coordinates not found for the specified city.


In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd

def scrape_gaming_laptops():
    driver = webdriver.Chrome()  # Make sure you have chromedriver installed and in PATH
    driver.get("https://www.digit.in/top-products/best-gaming-laptops-40.html")

    laptops = []
    laptop_list = driver.find_elements(By.CLASS_NAME, "TopNumbeHeading")
    for laptop in laptop_list:
        details = {}
        details["Name"] = laptop.find_element(By.CLASS_NAME, "TopNumbeHeadingname").text.strip()
        details["Price"] = laptop.find_element(By.CLASS_NAME, "smprice").text.strip()
        details["OS"] = laptop.find_elements(By.CLASS_NAME, "Spcs-details")[0].find_elements(By.CLASS_NAME, "Spcs-key")[1].text.strip()
        details["Display"] = laptop.find_elements(By.CLASS_NAME, "Spcs-details")[1].find_elements(By.CLASS_NAME, "Spcs-key")[1].text.strip()
        details["Processor"] = laptop.find_elements(By.CLASS_NAME, "Spcs-details")[2].find_elements(By.CLASS_NAME, "Spcs-key")[1].text.strip()
        details["Memory"] = laptop.find_elements(By.CLASS_NAME, "Spcs-details")[3].find_elements(By.CLASS_NAME, "Spcs-key")[1].text.strip()
        details["Weight"] = laptop.find_elements(By.CLASS_NAME, "Spcs-details")[4].find_elements(By.CLASS_NAME, "Spcs-key")[1].text.strip()
        details["Description"] = laptop.find_element(By.CLASS_NAME, "Section-center").text.strip()
        laptops.append(details)

    driver.quit()
    return laptops

def save_to_csv(data, filename):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)
    print(f"Data saved to {filename}")

if __name__ == "__main__":
    data = scrape_gaming_laptops()
    save_to_csv(data, "best_gaming_laptops.csv")


Data saved to best_gaming_laptops.csv


In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

def scrape_billionaires():
    driver = webdriver.Chrome()  # Make sure you have chromedriver installed and in PATH
    url = "https://www.forbes.com/billionaires/"
    driver.get(url)

    billionaires = []

    # Find and click the "Accept" button for cookies
    accept_button = driver.find_element(By.XPATH, "//button[@data-testid='cookie-policy-dialog-accept-button']")
    accept_button.click()

    # Keep clicking the "Show more" button until it's not found
    while True:
        try:
            show_more_button = driver.find_element(By.XPATH, "//button[contains(@class, 'expander-icon-container')]")
            show_more_button.click()
            time.sleep(1)  # Allow time for more profiles to load
        except:
            break

    # Find all billionaire profiles
    profiles = driver.find_elements(By.XPATH, "//div[@class='rank']")
    for profile in profiles:
        details = {}
        details["Rank"] = profile.find_element(By.XPATH, "./div[@class='rank']").text.strip()
        details["Name"] = profile.find_element(By.XPATH, "./div[@class='personName']").text.strip()
        details["Net worth"] = profile.find_element(By.XPATH, "./div[@class='netWorth']").text.strip()
        details["Age"] = profile.find_element(By.XPATH, "./div[@class='age']").text.strip()
        details["Citizenship"] = profile.find_element(By.XPATH, "./div[@class='countryOfCitizenship']").text.strip()
        details["Source"] = profile.find_element(By.XPATH, "./div[@class='source']").text.strip()
        details["Industry"] = profile.find_element(By.XPATH, "./div[@class='category']").text.strip()
        billionaires.append(details)

    driver.quit()
    return billionaires

def save_to_csv(data, filename):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)
    print(f"Data saved to {filename}")

if __name__ == "__main__":
    data = scrape_billionaires()
    save_to_csv(data, "forbes_billionaires.csv")


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[@data-testid='cookie-policy-dialog-accept-button']"}
  (Session info: chrome=123.0.6312.123); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF66BBD7032+63090]
	(No symbol) [0x00007FF66BB42C82]
	(No symbol) [0x00007FF66B9DEC65]
	(No symbol) [0x00007FF66BA2499D]
	(No symbol) [0x00007FF66BA24ADC]
	(No symbol) [0x00007FF66BA65B37]
	(No symbol) [0x00007FF66BA4701F]
	(No symbol) [0x00007FF66BA63412]
	(No symbol) [0x00007FF66BA46D83]
	(No symbol) [0x00007FF66BA183A8]
	(No symbol) [0x00007FF66BA19441]
	GetHandleVerifier [0x00007FF66BFD25AD+4238317]
	GetHandleVerifier [0x00007FF66C00F70D+4488525]
	GetHandleVerifier [0x00007FF66C0079EF+4456495]
	GetHandleVerifier [0x00007FF66BCB0576+953270]
	(No symbol) [0x00007FF66BB4E54F]
	(No symbol) [0x00007FF

In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

def scrape_youtube_comments(video_url, num_comments):
    driver = webdriver.Chrome()  # Make sure you have chromedriver installed and in PATH
    driver.get(video_url)
    time.sleep(5)  # Give time for the page to load

    # Scroll down to load comments
    SCROLL_PAUSE_TIME = 5
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Extract comments, upvotes, and time posted
    comments = []
    comment_elements = driver.find_elements(By.CSS_SELECTOR, "#content-text")
    upvote_elements = driver.find_elements(By.CSS_SELECTOR, "#vote-count-middle")
    time_elements = driver.find_elements(By.CSS_SELECTOR, "#header-author > yt-formatted-string > a > #published-time-text")

    for comment, upvote, time_element in zip(comment_elements, upvote_elements, time_elements):
        comment_text = comment.text.strip()
        upvote_count = upvote.text.strip()
        time_posted = time_element.get_attribute("aria-label").split(" ")[-2:]
        comments.append({"Comment": comment_text, "Upvotes": upvote_count, "Time Posted": " ".join(time_posted)})

        if len(comments) >= num_comments:
            break

    driver.quit()
    return comments

if __name__ == "__main__":
    video_url = input("Enter the YouTube video URL: ")
    num_comments = 500
    comments = scrape_youtube_comments(video_url, num_comments)
    for comment in comments:
        print(comment)


Enter the YouTube video URL: wheels on bus


InvalidArgumentException: Message: invalid argument
  (Session info: chrome=123.0.6312.123)
Stacktrace:
	GetHandleVerifier [0x00007FF66BBD7032+63090]
	(No symbol) [0x00007FF66BB42C82]
	(No symbol) [0x00007FF66B9DEB1D]
	(No symbol) [0x00007FF66B9CC76D]
	(No symbol) [0x00007FF66B9CB100]
	(No symbol) [0x00007FF66B9CB8BC]
	(No symbol) [0x00007FF66B9E14AD]
	(No symbol) [0x00007FF66BA63D67]
	(No symbol) [0x00007FF66BA46FDA]
	(No symbol) [0x00007FF66BA63412]
	(No symbol) [0x00007FF66BA46D83]
	(No symbol) [0x00007FF66BA183A8]
	(No symbol) [0x00007FF66BA19441]
	GetHandleVerifier [0x00007FF66BFD25AD+4238317]
	GetHandleVerifier [0x00007FF66C00F70D+4488525]
	GetHandleVerifier [0x00007FF66C0079EF+4456495]
	GetHandleVerifier [0x00007FF66BCB0576+953270]
	(No symbol) [0x00007FF66BB4E54F]
	(No symbol) [0x00007FF66BB49224]
	(No symbol) [0x00007FF66BB4935B]
	(No symbol) [0x00007FF66BB39B94]
	BaseThreadInitThunk [0x00007FF8430B7344+20]
	RtlUserThreadStart [0x00007FF8444C26B1+33]


In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

def scrape_youtube_comments(video_url, num_comments):
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-extensions')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')

    driver = webdriver.Chrome(options=options)  # Use headless mode to speed up scraping
    driver.get(video_url)
    time.sleep(5)  # Give time for the page to load

    # Scroll down to load comments
    SCROLL_PAUSE_TIME = 5
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Extract comments, upvotes, and time posted
    comments = []
    comment_elements = driver.find_elements(By.CSS_SELECTOR, "#content-text")
    upvote_elements = driver.find_elements(By.CSS_SELECTOR, "#vote-count-middle")
    time_elements = driver.find_elements(By.CSS_SELECTOR, "#header-author > yt-formatted-string > a > #published-time-text")

    for comment, upvote, time_element in zip(comment_elements, upvote_elements, time_elements):
        comment_text = comment.text.strip()
        upvote_count = upvote.text.strip()
        time_posted = time_element.get_attribute("aria-label").split(" ")[-2:]
        comments.append({"Comment": comment_text, "Upvotes": upvote_count, "Time Posted": " ".join(time_posted)})

        if len(comments) >= num_comments:
            break

    driver.quit()
    return comments

if __name__ == "__main__":
    video_url = input("Enter the YouTube video URL: ")
    num_comments = 500
    comments = scrape_youtube_comments(video_url, num_comments)
    for comment in comments:
        print(comment)


Enter the YouTube video URL: http://www.youtube.com/watch?v=-wtIMTCHWuI


In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd

def scrape_hostels_in_london():
    driver = webdriver.Chrome()  # Make sure you have chromedriver installed and in PATH
    url = "https://www.hostelworld.com/hostels/London/England"
    driver.get(url)

    hostels = []

    try:
        hostel_list = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "property-card")))
    except TimeoutException:
        print("Timeout occurred while waiting for hostels to load")
        driver.quit()
        return []

    for hostel in hostel_list:
        details = {}
        details["Hostel Name"] = hostel.find_element(By.CLASS_NAME, "title").text.strip()

        distance_elem = hostel.find_element(By.CLASS_NAME, "description")
        details["Distance from City Centre"] = distance_elem.text.strip()

        ratings_elem = hostel.find_element(By.CLASS_NAME, "score")
        details["Ratings"] = ratings_elem.text.strip()

        reviews_elem = hostel.find_element(By.CLASS_NAME, "reviews")
        details["Total Reviews"] = reviews_elem.text.strip().split()[0]

        overall_reviews_elem = hostel.find_element(By.CLASS_NAME, "keyword")
        details["Overall Reviews"] = overall_reviews_elem.text.strip()

        price_elements = hostel.find_elements(By.CLASS_NAME, "price-col")
        details["Privates from Price"] = price_elements[0].text.strip() if price_elements else "-"
        details["Dorms from Price"] = price_elements[1].text.strip() if len(price_elements) > 1 else "-"

        facilities_elem = hostel.find_element(By.CLASS_NAME, "facilities-label")
        details["Facilities"] = ", ".join([item.text.strip() for item in facilities_elem.find_elements(By.TAG_NAME, "span")])

        description_elem = hostel.find_element(By.CLASS_NAME, "rating-factors")
        details["Property Description"] = description_elem.text.strip()

        hostels.append(details)

    driver.quit()
    return hostels

def save_to_csv(data, filename):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)
    print(f"Data saved to {filename}")

if __name__ == "__main__":
    data = scrape_hostels_in_london()
    if data:
        save_to_csv(data, "hostels_in_london.csv")
    else:
        print("No hostels data retrieved.")


Timeout occurred while waiting for hostels to load
No hostels data retrieved.
